In [1]:
import pandas as pd
import win32com.client
import os
from datetime import datetime
import importlib
import warnings

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 7)
# update_files = True
update_files = True

warnings.filterwarnings('ignore')

# Get SSO of User
sso_user = os.path.expanduser("~")[-9:]

In [2]:
def import_salesforce_data(update_files=True):
    file_address = os.path.join(
        r"C:/Users",
        sso_user,
        "Box",
        "FieldCore Technical Learning and Development",
        "Tools",
        "RAW Data for Reports and Tools",
        "data for pythong scripts",
        "salesforce_data.xlsx",
    )

    print(f"Reading in file from {file_address}")

    if update_files:
        """Returns True if the file was refreshed today, False otherwise."""
        last_modified_time = os.path.getmtime(file_address)
        today = datetime.today()

        if not today.date() == datetime.fromtimestamp(last_modified_time).date():
            print("Updaing Smax File")
            updated_smax(file_address)

        else:
            refresh_input = input("Do you want to refresh smax (y/n)?")
            if refresh_input == "y":
                updated_smax(file_address)
            else:
                print("No update needed")

    if "dfs" in locals() or "dfs" in globals():
        print("Salesforce data already loaded")
    else:
        # read in salesforce data
        print("Reading in Salesforce Data...")
        try:
            excel_file = pd.ExcelFile(file_address)
            print("Salesforce Data loaded...")
        except FileNotFoundError:
            print(f"Error: file not found at '{file_address}'")
        except pd.errors.ParserError:
            print(
                f"Error: Failed to parse the finance data CSV file at '{file_address}'"
            )
        except Exception as e:
            print(f"Error: {str(e)}")

        sheet_names = excel_file.sheet_names
        dfs = {sheet_name: excel_file.parse(sheet_name) for sheet_name in sheet_names}

        excel_file.close()
        print("...complete")

    return dfs

def updated_smax(file_address):
    # Opening Excel software using the win32com
    File = win32com.client.Dispatch("Excel.Application")

    # Optional line to show the Excel software
    File.Visible = 1

    # Opening your workbook
    print("Updating Smax File")
    Workbook = File.Workbooks.open(file_address)

    # Refeshing all the shests
    Workbook.RefreshAll()
    File.CalculateUntilAsyncQueriesDone()

    # Saving the Workbook
    Workbook.Save()
    Workbook.Close()
    # Closing the Excel File
    File.Quit()


In [3]:
if 'dfs' in locals() or 'dfs' in globals():
    print("Salesforce data already loaded")
else:
    dfs = import_salesforce_data(update_files=False)

Reading in file from C:/Users\605042670\Box\FieldCore Technical Learning and Development\Tools\RAW Data for Reports and Tools\data for pythong scripts\salesforce_data.xlsx
Reading in Salesforce Data...
Salesforce Data loaded...
...complete


In [32]:
grand_Total_df = dfs["Grand Total Ops"]
grand_Total_df.to_excel("C:/Users/605042670/Desktop/Github/Training_Operations_Bulletin/2_pipeline/ServiceMax_GrandTotal.xlsx")
destination_df = pd.read_excel("C:/Users/605042670/Desktop/Github/Training_Operations_Bulletin/2_pipeline/GrandTotal.xlsx", sheet_name=None)

In [33]:
# Specify the Excel file path
excel_file_path = "C:/Users/605042670/Desktop/Github/Training_Operations_Bulletin/2_pipeline/GrandTotal.xlsx"

# Get the current week number
current_week = datetime.now().isocalendar()[1]

# Use pd.read_excel with sheet_name set to None to get the sheet names
excel_sheets = pd.read_excel(excel_file_path, sheet_name=None)

# Get the sheet names
sheet_names = list(excel_sheets.keys())

# Specify the Excel file paths
source_file_path = "C:/Users/605042670/Desktop/Github/Training_Operations_Bulletin/2_pipeline/ServiceMax_GrandTotal.xlsx"
# Read source data
source_df = pd.read_excel(source_file_path)

# Read destination data into a dictionary of DataFrames
destination_sheets = pd.read_excel(excel_file_path, sheet_name=None)

# Iterate over each sheet in the destination file
for sheet_name, destination_df in destination_sheets.items():
    # Check if the sheet name exists in the 'Region' column of the source file
    if sheet_name in source_df['Region'].values:
        # Filter source data based on the 'Region' and current week number
        source_data_for_sheet = source_df[(source_df['Region'] == sheet_name) & (source_df['Week'] == current_week)][['Ops#1', 'Ops#2', 'Ops#3', 'Ops#4', 'Ops#5', 'Ops#6', 'Ops#7', 'Ops#8', 'Ops#9', 'Ops#10', 'Ops#11', 'Ops#12']]

        # Merge or append the additional data to the existing sheet data
        updated_df = pd.merge(destination_df, source_data_for_sheet, left_on='Week', right_index=True, how='left', suffixes=('', '_source'))

        # Write the updated DataFrame to the sheet
        updated_df.to_excel(excel_file_path, sheet_name=sheet_name, index=False)
    else:
        print(f"Sheet '{sheet_name}' not found in the source file.")

print("Update completed.")


KeyError: 'Week'

,Week,Ops#1,Ops#2,Ops#3,Ops#4,Ops#5,Ops#6,Ops#7,Ops#8,Ops#9,Ops#10,Ops#11,Ops#12
0,week 46,4663.0,11.0,4.0,2.0,0.0,267.0,12.0,NaN,NaN,0.0,70.0,0.0
1,week 47,4632.0,11.0,2.0,152.0,0.0,159.0,12.0,NaN,NaN,0.0,70.0,0.0
2,week 48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,week 09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,week 10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,week 11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Get the week of the year
current_date = datetime.now()
week_number = current_date.isocalendar()[1]
print(f"Week of the year: {week_number}")


In [ ]:
# Iterate over each sheet in the source file
for sheet_name, source_sheet in grand_Total_df.groupby('Region'):
    # Check if the sheet exists in the destination file
    if sheet_name in destination_df:
        # Merge the source data into the destination data based on Week
        merged_data = pd.merge(destination_df[sheet_name], source_sheet[['Week'] + [f'Ops#{i}' for i in range(1, 13)]], on='Week', how='outer')

        # Sort the data based on Week
        merged_data.sort_values(by='Week', inplace=True)

        # Update the destination sheet with the merged and sorted data
        destination_df[sheet_name] = merged_data
    else:
        print(f"Sheet '{sheet_name}' not found in the destination file.")

# Save the updated destination data to the same Excel file
with pd.ExcelWriter("C:/Users/605042670/Desktop/Github/Training_Operations_Bulletin/2_pipeline\GrandTotal.xlsx", engine='openpyxl') as writer:
    for sheet_name, df in destination_df.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)